In [1]:
%autosave 1
import os
import argparse
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.patches import Rectangle
from z3 import *

Autosaving every 1 seconds


In [374]:
instance_name_int = 39
instance_name_str = str(instance_name_int) + 'x' + str(instance_name_int) + '.txt'
base_dir = 'Instances/'
with open(base_dir + instance_name_str) as f:
    content = f.readlines()
content = [x.strip() for x in content]
# remove empty strings
content = [x for x in content if x]
width = int(content[0].split(' ')[0])
height = int(content[0].split(' ')[1])
blobs_count = int(content[1])
blobs = []
for i in range(2, len(content)):
    blobs.append([int(content[i].split(' ')[0]), int(content[i].split(' ')[1])])

In [375]:
# variables: array composed by pairs of (x, y) coordinates which are the starting point of the 
# respsective blobs in blobs array. This is a variable so it is not itialized yet, 
# is what we have to find.
O = [ [ Int("o_{}_{}".format(i+1, j+1)) for j in range(2) ] for i in range(blobs_count) ]

In [376]:
# constraint for ensure that the domain of the origins should be [0, width[ and [0, height[
in_domain = [ And(O[i][0] >= 0, O[i][0] < width, O[i][1] >= 0, O[i][1] < height)
             for i in range(blobs_count)] 

In [377]:
# each piece must fit in the paper roll
in_paper = [ And(blobs[i][0] + O[i][0] <= width, blobs[i][1] + O[i][1] <= height) 
            for i in range(blobs_count)] 

In [378]:
# non-overlap constraint using global constrint
no_overlap = []
for i in range(blobs_count):
    for j in range(blobs_count):
        if (i<j):
            no_overlap.append(Or(O[i][0] + blobs[i][0]  <= O[j][0], O[j][0] + blobs[j][0] <= O[i][0], O[i][1] + blobs[i][1] <= O[j][1], O[j][1] + blobs[j][1] <= O[i][1]))

In [379]:
# implied cumulative constraints
implied = []
for i in range(width):
    for j in range(blobs_count):
        implied.append(Sum([If(And(O[j][0] <= i, i < O[j][0] + blobs[j][0]), blobs[j][1],0) for j in range(blobs_count)]) <= height)

for i in range(height):
    for j in range(blobs_count):
        implied.append(Sum([If(And(O[j][1] <= i, i < O[j][1] + blobs[j][1]), blobs[j][0],0) for j in range(blobs_count)]) <= width)

### Simmetry breaking

In [380]:
#this constraint forces the first (biggest area object) to be placed into thefirst, bottom-left quadrant of the box
biggest_obj_bottom_left = [ And(O[0][0] <= (width/2 - blobs[0][0]/2), O[0][1] <= (height/2 - blobs[0][1]/2))] 

In [381]:
# TODO: syntax error
# identical rectangle constraint
#identical = [If(And(blobs[i][0] == blobs[i+1][0], blobs[i][1] == blobs[i+1][1]), 
#   Or(O[i][0] < O[i+1][0], And(O[i][0] == O[i+1][0], O[i][1] < O[i+1][1])),
#   False) for i in range(blobs_count)]

In [382]:
constraints = in_domain + in_paper + no_overlap + biggest_obj_bottom_left

In [383]:
s = Solver()
s.add(constraints)

In [384]:
if s.check() == sat:
    m = s.model()

    print("{} {}".format(height, width))
    print("{}".format(blobs_count))

    for i in range(blobs_count):
        print("{:<1} {:<3} {:<1} {:<2}".format(blobs[i][0], blobs[i][1], str(m[O[i][0]]), str(m[O[i][1]])))
    print("\n{}\n".format(s.statistics()))
else: 
    print("Failed to solve")

Failed to solve
